Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
`https://github.com/squillero/computational-intelligence`  
Free for personal or classroom use; see 'LICENCE.md' for details.

In [153]:
import logging
import numpy as np

from collections import deque

logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s', datefmt='%H:%M:%S', level=logging.INFO)

In [154]:
def _contains_duplicates(X):
    return np.sum(np.unique(X)) != np.sum(X)

def contains_duplicates(sol):
    return any(_contains_duplicates(sol[r,:]) for r in range(9)) or \
           any(_contains_duplicates(sol[:,r]) for r in range(9)) or \
           any(_contains_duplicates(sol[r:r+3:,c:c+3]) for r in range(0,9,3) for c in range(0,9,3))

def valid_solution(sol):
    return not contains_duplicates(sol) and np.sum(sol) == (1+2+3+4+5+6+7+8+9) * 9

def forward_checking_valid(sudoku, possible_values, i, j):
    # ROW
    mask = np.array([[False for j in range(9)] for i in range(9)])
    mask_r = sudoku[i, :] == 0
    mask[i, :] = mask_r
    # COL
    mask_c = sudoku[:, j] == 0
    mask[:, j] = mask_c
    # MACRO-CELL
    cell_r = i // 3
    cell_c = j // 3
    mask_m = sudoku[cell_r * 3 : cell_r * 3 + 3, cell_c * 3 : cell_c * 3 + 3] == 0
    mask[cell_r * 3 : cell_r * 3 + 3, cell_c * 3 : cell_c * 3 + 3] = mask_m
    if any(mask.ravel()) and any(possible_values[mask].sum(axis=-1) == 0):
        return False

    return True


def print_sudoku(sudoku):
    print("+-------+-------+-------+")
    for b in range(0, 9, 3):
        for r in range(3):
            print("|", " | ".join(" ".join(str(_) for _ in sudoku[b+r, c:c+3]) for c in range(0, 9, 3)), "|")
        print("+-------+-------+-------+")

In [155]:
def dfsolve(sudoku):
    """Vanilla depth-first solver for sudoku puzzles"""
    frontier = deque([sudoku.copy()])
    num_nodes = 0
    while frontier:
        node = frontier.popleft()
        num_nodes += 1

        if valid_solution(node):
            logging.info(f"Solved after expanding {num_nodes:,} nodes")
            return node

        for i, j in zip(*np.where(node == 0)):
            for c in range(1, 10):
                node[i, j] = c
                if not contains_duplicates(node):
                    frontier.appendleft(node.copy())
    logging.info(f"Giving up after expanding {num_nodes:,} nodes")
    return None

In [156]:
def remove_possible_value(possible_values, r, c, element):
    element -= 1
    possible_values[:, c, element] = 0
    possible_values[r, :, element] = 0
    cell_r = r // 3
    cell_c = c // 3
    possible_values[cell_r * 3 : cell_r * 3 + 3, cell_c * 3 : cell_c * 3 + 3, element] = 0

    return possible_values

def compute_possible_remaining_values(sudoku):
    possible_values = np.ones((9, 9, 9), dtype=np.int8)
    for i, j in zip(*np.where(sudoku != 0)):
            element = sudoku[i, j]

            remove_possible_value(possible_values, i, j, element)
    
    return possible_values

def least_constraining_value_sorted(sudoku, possible_vals, i, j):
    vals = [[v, 0] for v in range(1, 10)]
    sudoku = sudoku.copy()
    for i, v in enumerate(vals):
        sudoku[i, j] = v[0]
        new_possible_vals = compute_possible_remaining_values(sudoku)
        poss_vals = new_possible_vals.sum()
        vals[i][1] = poss_vals
    
    vals = sorted(vals, key=lambda v: v[1])
    vals = [v[0] for v in vals]
    return vals

def least_possible_values_sorted(sudoku, possible_vals):
    possible_vals = possible_vals.copy()
    possible_vals = possible_vals.sum(axis=-1)
    ind = np.unravel_index(np.argsort(possible_vals, axis=None), possible_vals.shape)
    return ind

def print_possible_values(possible_values):
    print("Possible values:")
    print("-------------------------------------------------------------------------")
    for r in range(9):
        for c in range(9):
            print(f"({r}, {c}):", end='')
            for poss_val in (np.where(possible_values[r, c] == 1))[0]:
                print(f"{poss_val+1}/", end='')
            print("\t", end="")
        print()
    print("-------------------------------------------------------------------------")
    print("")

In [157]:
def df_improved_solve(sudoku):
    """Vanilla depth-first solver for sudoku puzzles"""
    frontier = deque([sudoku.copy()])
    num_nodes = 0
    while frontier:
        node = frontier.popleft()
        num_nodes += 1

        if (node != 0).all() and valid_solution(node):
            logging.info(f"Solved after expanding {num_nodes:,} nodes")
            return node

        # Selecting the next cell to expand in order of
        # how many possible values can be put in that cell..
        # It slows down a lot the computation!
        
        #possible_values = compute_possible_remaining_values(sudoku)
        #sorted_indices = least_possible_values_sorted(sudoku, possible_values)
        #for i, j in zip(sorted_indices[0][::-1], sorted_indices[1][::-1]):
        #    if sudoku[i, j] != 0:
        #        continue
        for i, j in zip(*np.where(node == 0)):

            # Least constraining values.. it slows down the computation!
            #possible_values = compute_possible_remaining_values(node)
            # Increasing order.. (the least constraining will come at the end and in fact will be put on the left of the deque)
            #sorted_vals = least_constraining_value_sorted(node, possible_values, i, j)
            #print(f"Trying against {sorted_vals} for the cell i: {i}, j: {j}")
            #for c in sorted_vals:
            for c in range(1, 10):
                node[i, j] = c
                if not contains_duplicates(node):
                    possible_values = compute_possible_remaining_values(node)
                    if forward_checking_valid(node, possible_values, i, j) is False:
                        continue
                    frontier.appendleft(node.copy())
    logging.info(f"Giving up after expanding {num_nodes:,} nodes")
    return None

In [158]:
simple_sudoku = np.array([[6, 0, 4,    0, 7, 0,    0, 0, 1],
                          [0, 5, 0,    0, 0, 0,    0, 7, 0], 
                          [7, 0, 0,    5, 9, 6,    8, 3, 4], 
       
                          [0, 8, 0,    0, 0, 2,    4, 9, 0], 
                          [1, 0, 0,    0, 0, 0,    0, 0, 3], 
                          [0, 6, 9,    7, 0, 0,    0, 5, 0], 
       
                          [9, 1, 8,    3, 6, 7,    0, 0, 5], 
                          [0, 4, 0,    0, 0, 0,    0, 6, 0], 
                          [2, 0, 0,    0, 5, 0,    7, 0, 8]], dtype=np.int8)

In [159]:
df_improved_solve(simple_sudoku)

[23:43:37] INFO: Solved after expanding 219 nodes


array([[6, 9, 4, 8, 7, 3, 5, 2, 1],
       [8, 5, 3, 1, 2, 4, 9, 7, 6],
       [7, 2, 1, 5, 9, 6, 8, 3, 4],
       [3, 8, 5, 6, 1, 2, 4, 9, 7],
       [1, 7, 2, 9, 4, 5, 6, 8, 3],
       [4, 6, 9, 7, 3, 8, 1, 5, 2],
       [9, 1, 8, 3, 6, 7, 2, 4, 5],
       [5, 4, 7, 2, 8, 1, 3, 6, 9],
       [2, 3, 6, 4, 5, 9, 7, 1, 8]], dtype=int8)

In [160]:
def sudoku_generator(sudokus=1, *, kappa=5, random_seed=None):
    if random_seed:
        np.random.seed(random_seed)
    for puzzle in range(sudokus):
        sudoku = np.zeros((9, 9), dtype=np.int8)
        for cell in range(np.random.randint(kappa)):
            for p, val in zip(np.random.randint(0, 8, size=(9, 2)), range(1, 10)):
                tmp = sudoku.copy()
                sudoku[tuple(p)] = val
                if contains_duplicates(sudoku):
                    sudoku = tmp
        yield sudoku.copy()

In [163]:
for i, sudoku in enumerate(sudoku_generator(sudokus=4, random_seed=42)):
    print(f"Sudoku {i+1}:")
    print_sudoku(sudoku)
    print("[MY SOLUTION] Searching a solution with depth-first improved solver..")
    solution = df_improved_solve(sudoku)
    if (solution is not None):
        print("[MY SOLUTION] Solution with depth-first improved solver found.\n")
    print("[PROF SOLUTION] Searching a solution with depth-first solver..")
    solution_df = dfsolve(sudoku)
    if solution_df is not None:
        print("[PROF SOLUTION] Solution with depth-first solver found.")
    
    print("--------------------------------------------------\n")

Sudoku 1:
+-------+-------+-------+
| 0 0 1 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 3 0 |
| 0 0 6 | 8 0 0 | 5 2 0 |
+-------+-------+-------+
| 9 7 0 | 4 0 0 | 0 8 0 |
| 6 0 0 | 0 3 0 | 1 0 0 |
| 0 0 0 | 0 8 6 | 0 0 0 |
+-------+-------+-------+
| 0 4 0 | 9 0 0 | 0 0 0 |
| 0 0 9 | 5 7 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+
[MY SOLUTION] Searching a solution with depth-first improved solver..


[23:50:43] INFO: Solved after expanding 410 nodes


[MY SOLUTION] Solution with depth-first improved solver found.

[PROF SOLUTION] Searching a solution with depth-first solver..


[23:51:19] INFO: Solved after expanding 1,735 nodes


[PROF SOLUTION] Solution with depth-first solver found.
--------------------------------------------------

Sudoku 2:
+-------+-------+-------+
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+
[MY SOLUTION] Searching a solution with depth-first improved solver..


...

[PROF SOLUTION] Solution with depth-first solver found.
--------------------------------------------------

Sudoku 4:
+-------+-------+-------+
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 3 0 | 0 0 5 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+
| 0 4 0 | 0 0 6 | 0 0 0 |
| 0 0 0 | 0 0 0 | 2 0 0 |
| 0 0 0 | 0 0 0 | 9 0 0 |
+-------+-------+-------+
| 0 0 0 | 0 0 0 | 0 8 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+
[MY SOLUTION] Searching a solution with depth-first improved solver..


[23:51:44] INFO: Solved after expanding 100 nodes


[MY SOLUTION] Solution with depth-first improved solver found.

[PROF SOLUTION] Searching a solution with depth-first solver..


[23:51:49] INFO: Solved after expanding 263 nodes


[PROF SOLUTION] Solution with depth-first solver found.
--------------------------------------------------

